In [1]:
from importlib import reload  # Python 3.4+
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from ndbc_api import NdbcApi
import concurrent.futures
from tqdm.contrib.concurrent import process_map, thread_map
from tqdm.autonotebook import tqdm
import xbuoy
import matplotlib
import seaborn as sns


In [6]:
extrema = lambda x: np.array((x.min().values, x.max().values))

buoy_ds = xr.open_dataset("NDBC_CC_Daily.nc")
buoy_ds = buoy_ds.resample(time="1W").mean("time")
# buoy_ds = buoy_ds.resample(time="1M").mean("time")
buoy_ds["WTMP"] = buoy_ds["WTMP"] - buoy_ds["WTMP"].mean("time")
buoy_ds["WTMP_loc_avg"] = buoy_ds["WTMP"].mean("station_id")
buoy_ds["WTMP_loc_spread"] = buoy_ds["WTMP"].std("station_id")
wtmp_extrema = extrema(buoy_ds["WTMP"]).max().round() / 1.5

# buoy_ds = buoy_ds.isel(time = slice(1000, 1050))

In [7]:
import warnings
import cartopy.feature as cfeature
import cmocean.cm as cm
import cmocean
from shapely.errors import ShapelyDeprecationWarning
from xmovie import Movie
import seaborn as sns
sns.set_context("notebook")

warnings.filterwarnings(
    action='ignore',
    category=ShapelyDeprecationWarning,  # in cartopy
)
warnings.filterwarnings(
    action="ignore",
    category=UserWarning)


def custom_plotfunc(ds, fig, tt, *args, **kwargs):
    buoy_ds = ds
    # ax = fig.subplots(ncols=1, subplot_kw={'projection':ccrs.PlateCarree()})
    ax = fig.add_subplot(1, 2, 2, projection=ccrs.PlateCarree())

    ax.set_aspect(0.75)

    fig.tight_layout()

    # Plot each station's location as a scatter point
    for station in buoy_ds.station_id:
        station_data = buoy_ds.isel(time = tt).sel(station_id = station)
        cb = ax.scatter(station_data.longitude, station_data.latitude, c = station_data["WTMP"], 
                   vmin = -wtmp_extrema, vmax = wtmp_extrema,
                   edgecolors='black', cmap = cm.balance,
                   transform=ccrs.PlateCarree())

    # Convert string to datetime object
    formatted_date = buoy_ds.isel(time = tt).time.values.astype('datetime64[D]').astype(str)
    # Format as YYYY-MM-DD

    ax.set_title(str(formatted_date));
    fig.tight_layout()


    # Add coastlines to the map with some transparency
    ax.coastlines(alpha=0.5)
    ax.add_feature(cfeature.LAND, color = "#d3d3d3")

    # Add gridlines with labels on the map, disabling top and right labels
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0.1, color = "grey")
    gl.top_labels = False
    gl.right_labels = False

    fig.colorbar(cb, label = "Temperature Anomaly [deg C]", extend = "both", ax = ax, orientation = "vertical", fraction = 0.035)

    ax2 = fig.add_subplot(1, 2, 1)
    ax2.plot(buoy_ds.time, buoy_ds["WTMP_loc_avg"], color = "orange")
    ax2.fill_between(buoy_ds.time, 
                    buoy_ds["WTMP_loc_avg"] - 2 * buoy_ds["WTMP_loc_spread"],  
                    buoy_ds["WTMP_loc_avg"] + 2 * buoy_ds["WTMP_loc_spread"], 
                   alpha = 0.3, color = "orange")
    ax2.scatter(buoy_ds.isel(time = tt).time, buoy_ds.isel(time = tt)["WTMP_loc_avg"], c = "k", s = 20, zorder = 10)
    ax2.set_title("Buoy Water Temperature Anomaly")
    ax2.set_ylabel("deg C")
    ax2.set_xlabel("time")
    ax2.tick_params(axis='x', rotation=45)

    # ax2.set_xticks(rotation = 45)
    fig.tight_layout()
    fig.subplots_adjust(bottom = 0.3)
    return None, None
    # ^ This is not strictly necessary, but otherwise a warning will be raised.

mov_custom = Movie(buoy_ds.chunk({'time':1}), custom_plotfunc, input_check = False)

In [ ]:
mov_custom.save('buoy_temperature_anomalies.mp4', progress=True, 
         overwrite_existing = True,parallel=True, framerate=20,
       parallel_compute_kwargs=dict(scheduler="processes", num_workers=4)
)

/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/pyproj/transformer.py:820: UserWarning: Warning: converting a masked element to nan.
  return self._transformer._transform_point(
/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/pyproj/transformer.py:820: UserWarning: Warning: converting a masked element to nan.
  return self._transformer._transform_point(
/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/pyproj/transformer.py:820: UserWarning: Warning: converting a masked element to nan.
  return self._transformer._transform_point(
/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/pyproj/transformer.py:820: UserWarning: Warning: converting a masked element to nan.
  return self._transformer._transform_point(
/Users/anthonymeza/miniforge3/lib/python3.10/site-packages/pyproj/transformer.py:820: UserWarning: Warning: converting a masked element to nan.
  return self._transformer._transform_point(
/Users/anthonymeza/miniforge3/lib/python3.10/site-packa